In [2]:
from Adapt import AdaptVQE_Y

from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureMoleculeDriver, ElectronicStructureDriverType
from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer
import warnings
from qiskit_nature.circuit.library import HartreeFock,UCCSD
from qiskit import Aer
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 
warnings.filterwarnings("ignore")


In [3]:
molecule = Molecule(
    # coordinates are given in Angstrom
    geometry=[
        ["O", [0.0, 0.0, 0.0]],
        ["H", [0.758602, 0.0, 0.504284]],
        ["H", [0.758602, 0.0, -0.504284]]
    ],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)

driver = ElectronicStructureMoleculeDriver(
    molecule=molecule,
    basis="sto3g",
    driver_type=ElectronicStructureDriverType.PYSCF,
)

properties = driver.run()
converter = QubitConverter(JordanWignerMapper())
PN_property = properties.get_property("ParticleNumber")
print(PN_property)

# Define the active space around the Fermi level 
# (selected automatically around the HOMO and LUMO, ordered by energy)
transformer = ActiveSpaceTransformer(
    num_electrons=2, #how many electrons we have in our active space
    num_molecular_orbitals=2, #how many orbitals we have in our active space
)

problem = ElectronicStructureProblem(driver, [transformer])

ParticleNumber:
	14 SOs
	5 alpha electrons
		orbital occupation: [1. 1. 1. 1. 1. 0. 0.]
	5 beta electrons
		orbital occupation: [1. 1. 1. 1. 1. 0. 0.]


In [5]:
a = AdaptVQE_Y(es_problem=problem)
a.problem_second_op

/home/yangjianfei/miniconda3/envs/QISKIT_WSL2/lib/python3.10/site-packages/qiskit_nature/problems/second_quantization/electronic/electronic_structure_problem.py:93: ListAuxOpsDeprecationWarning: List-based `aux_operators` are deprecated as of version 0.3.0 and support for them will be removed no sooner than 3 months after the release. Instead, use dict-based `aux_operators`. You can switch to the dict-based interface immediately, by setting `qiskit_nature.settings.dict_aux_operators` to `True`.
  second_quantized_ops = self._grouped_property_transformed.second_q_ops()


[FermionicOp([('+_0 -_0', -1.287960526673277), ('+_1 -_1', -0.5749964325186947), ('+_2 -_2', -1.287960526673277), ('+_3 -_3', -0.5749964325186947), ('+_0 +_0 -_0 -_0', -0.44007954668752175), ('+_0 +_1 -_1 -_0...)], register_length=4, display_format='sparse'),
 FermionicOp([('+_0 -_0', (1+0j)), ('+_1 -_1', (1+0j)), ('+_2 -_2', (1+0j)), ('+_3 -_3', (1+0j))], register_length=4, display_format='sparse'),
 FermionicOp([('+_0 -_0', (0.75+0j)), ('+_1 -_1', (0.75+0j)), ('+_2 -_2', (0.75+0j)), ('+_3 -_3', (0.75+0j)), ('+_0 -_0 +_1 -_1', (0.5+0j)), ('+_0 -_0 +_2 -_2', (-1.5+0j)), ('+_0 -_0 +_3 -_3', (-0.5+0j)), ('+...)], register_length=4, display_format='sparse'),
 FermionicOp([('+_0 -_0', (0.5+0j)), ('+_1 -_1', (0.5+0j)), ('+_2 -_2', (-0.5+0j)), ('+_3 -_3', (-0.5+0j))], register_length=4, display_format='sparse'),
 FermionicOp([('+_1 -_1', 1.0944875768738576), ('+_3 -_3', 1.0944875768738576)], register_length=4, display_format='sparse'),
 FermionicOp([('+_0 -_1', 0.04973035940288941), ('+_1 -_

In [7]:
a.hamiltoinian
a.molecular_hamiltoninan

PauliSumOp(SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'ZIIZ', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.87206336+0.j,  0.11923749+0.j, -0.17865262+0.j,  0.11923749+0.j,
 -0.17865262+0.j,  0.14753433+0.j,  0.22003977+0.j,  0.00963433+0.j,
  0.00963433+0.j,  0.00963433+0.j,  0.00963433+0.j,  0.15716866+0.j,
  0.15716866+0.j,  0.16144783+0.j,  0.14753433+0.j]), coeff=1.0)

In [8]:
a.show()

2次量子化后的哈密顿量为:



PauliSumOp(SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'ZIIZ', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.87206336+0.j,  0.11923749+0.j, -0.17865262+0.j,  0.11923749+0.j,
 -0.17865262+0.j,  0.14753433+0.j,  0.22003977+0.j,  0.00963433+0.j,
  0.00963433+0.j,  0.00963433+0.j,  0.00963433+0.j,  0.15716866+0.j,
  0.15716866+0.j,  0.16144783+0.j,  0.14753433+0.j]), coeff=1.0)

In [ ]:
from qiskit.opflow import X